In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import seaborn as sns
import pandas as pd
import gensim
import nltk
import re
import string
from curses.ascii import isprint

In [4]:
stop_words = open('stop_words').read().split('\n')[:-1]
#stop_words = 'the,for,to,you,call,that'.split(',')
#print stop_words

In [5]:
emails_table = pd.read_csv("data/emails/Emails.csv").dropna()
person_table = pd.read_csv("data/emails/Persons.csv").dropna()
alias_table = pd.read_csv("data/emails/Aliases.csv").dropna()
email_rec_table = pd.read_csv("data/emails/EmailReceivers.csv").dropna()

In [6]:
emails_table.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
16,17,C05739573,THE YOUTH OF LIBYA,H,"Sherman, Wendy R",213.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739573...,F-2015-04841,...,"H; Mills, Cheryl D; Sullivan, Jacob J; Nuiand,...","Sherman, Wendy R <ShermanWR@state.gov>","Escrogirna, Ana A; Grantham, Chris W","Wednesday, September 12, 2012 9:57 PM",F-2015-04841,C05739573,05/13/2015,RELEASE IN FULL,Hope. See picture below Kamala sent.,"-ottgat,t\n4 44 4, 0\n.\n1.3.141.4443\nUNCLASS..."
17,18,C05739575,ONE MORE PHOTO,H,"Sherman, Wendy R",213.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739575...,F-2015-04841,...,"H; Mills, Cheryl D; Sullivan, Jacob J, Nuland,...","Sherman, Wendy R <ShermanWR@state.gov>","Escrogima, Ana A","Wednesday, September 12, 2012 10:03 PM",F-2015-04841,C05739575,05/13/2015,RELEASE IN FULL,Another photo.,c. 371.14\nUNCLASSIFIED\nU.S. Department of St...
65,66,C05739638,BRITS AND LIBYA,sullivanjj@state.gov,H,80.0,2011-04-02T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739638...,F-2015-04841,...,'sullivanjj@state.gov',H <hrod17@clintonemailcom>,Huma Abedin,"Saturday, April 02, 2011 1034 AM",F-2015-04841,C05739638,05/13/2015,RELEASE IN PART,"When is Chris Stevens going?\n---,",UNCLASSIFIED\nU.S. Department of State\nCase N...
82,83,C05739656,THE CABLE: ROMNEY TO COMPARE BENGHAZI TO ORIGI...,millscd@state.gov,H,80.0,2012-10-08T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH3/DOC_0C05739656...,F-2015-04841,...,"'miliscd@stategov""",H <hrod17@clintonemail.com>,"Reines, Philippe 1; Mills, Cheryl D; Adams, Da...","Monday, October 08, 2012 07:26 AM",F-2015-04841,C05739656,05/13/2015,RELEASE IN FULL,Can you call me?,UNCLASSIFIED\nU.S. Department of State\nCase N...


In [7]:
person_table.head()

,Id,Name
0,1,111th Congress
1,2,AGNA USEMB Kabul Afghanistan
2,3,AP
3,4,ASUNCION
4,5,Alec


In [8]:
alias_table.head()

,Id,Alias,PersonId
0,1,111th congress,1
1,2,agna usemb kabul afghanistan,2
2,3,ap,3
3,4,asuncion,4
4,5,alec,5


In [9]:
email_rec_table.head()

,Id,EmailId,PersonId
0,1,1,80
1,2,2,80
2,3,3,228
3,4,3,80
4,5,4,80


In [10]:
print len(emails_table)
print len(person_table)
print len(alias_table)
print len(email_rec_table)

908
513
850
9306


In [11]:
emails_table['doc'] = emails_table["ExtractedSubject"] + " " + emails_table["ExtractedBodyText"]

In [12]:
def clean_text(text):
    ot = text
    # Remove any beginning or ending whitespace
    text.strip()
    # Combine conjunctions
    text = re.sub(r"'", "", text)
    # Replace punctuation with space
    text = text.translate(string.maketrans(string.punctuation,''.join(" " for char in string.punctuation)))
    # Replace all control characters
    text = ''.join(char if isprint(char) else " " for char in text)
    # Remove Digits
    text = re.sub(r"[0-9]", " ", text)
    
    
    text = text.lower()
    text_list = [token for token in text.split() if token not in stop_words and len(token) > 1]
    return text_list

In [13]:
emails_table['token_list'] = emails_table['doc'].apply(lambda s: clean_text(s))

In [14]:
dictionary = gensim.corpora.Dictionary(list(emails_table['token_list']))
dictionary.filter_extremes(2,.1)

In [15]:
emails_table[['SenderPersonId', 'Id']].head()

,SenderPersonId,Id
2,32.0,3
16,213.0,17
17,213.0,18
65,80.0,66
82,80.0,83


In [16]:
person_IDs = list(set(email_rec_table['PersonId']))
id2docs = {}
for pid in person_IDs:
    id2docs[pid] = list(emails_table[emails_table['SenderPersonId'] == pid]['token_list'])
len(id2docs[80])

505

In [17]:
print dictionary.num_docs
print dictionary.num_pos
print dictionary.num_nnz

908
9767
8921


In [18]:
corpus = [dictionary.doc2bow(text) for text in emails_table['token_list']]

In [19]:
#minP = 100
#currmin = 50
#for j in range(3):
#    for i in range(2,100):
#        lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=i)
#        newP = lda.log_perplexity(corpus)
#        if newP < minP:
#            currmin= i
#            minP = newP
#print currmin

#the above was a search over number of topics for the LDA model with lowest perplexity - the number found in 3 tries was 98


In [38]:
nt = 98
#only edges with weights above this threshold will be included.
w_thresh = .000000000005
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=nt)
#initialize complete matrix of empty weights. this probably isn't necessary
weights = np.zeros((len(person_IDs),nt))
#for each person
for i in range(len(person_IDs)):
    #for each document the person wrote
    for sublist in id2docs[person_IDs[i]]:
        #get any topics with probability higher than w_thresh for that document
        vals = lda.get_document_topics(dictionary.doc2bow(item for item in sublist),w_thresh)
        for pair in vals:
            weights[i][pair[0]] += pair[1]/len(id2docs[person_IDs[i]])

In [39]:
#these are Hillary's outgoing weights
print weights[person_IDs.index(80)]

[ 0.01105456  0.00859999  0.00873876  0.00821258  0.00822571  0.01486173
  0.00591548  0.00677211  0.01182547  0.01489245  0.00531828  0.00905079
  0.00657101  0.00955361  0.00751704  0.00883077  0.01153704  0.01153479
  0.0121366   0.00629379  0.01602722  0.01120701  0.01362101  0.01052051
  0.01030778  0.01472097  0.01061431  0.0082354   0.00927455  0.01052887
  0.0083912   0.01412706  0.00927804  0.01234665  0.00811572  0.00741427
  0.00907484  0.0083271   0.01131338  0.01034788  0.01096413  0.01530722
  0.00884784  0.01704014  0.02037215  0.01158732  0.00888263  0.01383044
  0.00508949  0.00940603  0.00763804  0.00672642  0.00879806  0.00990569
  0.00678355  0.01176649  0.01936871  0.00660025  0.01329761  0.01685546
  0.01087723  0.01010255  0.0157665   0.00649765  0.00793104  0.01214634
  0.00877746  0.01075804  0.00691078  0.01127137  0.0091443   0.00817091
  0.01562502  0.00769635  0.01172805  0.01017526  0.01083466  0.00672171
  0.00585444  0.00910815  0.00927083  0.01669193  0

[[  2.75483319e-03   2.48507928e-02   2.75483319e-03   2.75483319e-03
    5.46066850e-02   2.75483319e-03   2.75483319e-03   2.75483319e-03
    2.75483319e-03   2.52754833e-01   2.75483319e-03   2.75483319e-03
    4.40591810e-02   2.75483319e-03   2.75483319e-03   2.75483319e-03
    2.75483319e-03   2.75483319e-03   2.52754833e-01   2.75483319e-03
    2.75483319e-03   1.25203813e-01   2.75483319e-03   2.75483319e-03
    2.75483319e-03   2.75483319e-03   2.75483319e-03   2.75483319e-03
    2.75483319e-03   7.32676537e-02   2.75483319e-03   2.75483319e-03
    2.75483319e-03   2.75483319e-03   2.75483319e-03   1.27754833e-01
    2.75483319e-03   2.75483319e-03   2.75483319e-03   3.58582815e-02
    2.75483319e-03   2.75483319e-03   2.75483319e-03   2.75483319e-03
    2.75483319e-03   2.75483319e-03   2.75483319e-03   1.49813657e-01
    2.75483319e-03   2.75483319e-03   2.75483319e-03   2.75483319e-03
    1.27754833e-01   2.75483319e-03   2.75483319e-03   2.14514144e-02
    2.75483319e-03  